In [4]:
"""
Automatic speech recognition and speaker identity
"""
from __future__ import division, print_function, absolute_import
import os
import numpy as np
import sklearn.preprocessing as prep
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense, Dropout
from tensorflow.keras.layers import Embedding, Bidirectional,Conv1D,GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import utils,losses
print("tf=",tf.__version__)
print("keras=",keras.__version__)

tf= 2.2.0
keras= 2.3.0-tf


In [2]:
print("Imports doned")
def wav2mfcc(file_path, max_pad_len=30):
    wave, sampleRate = librosa.load(file_path, mono=True, sr=None)
    wave=prep.maxabs_scale(wave) #Normalize amplitude to +-1
    mfcc = librosa.feature.mfcc(wave, sr=sampleRate).transpose()
    print(sampleRate,"wave=",wave.shape,"mfcc=",mfcc.shape,"file=",file_path)
    #print(mfcc[1,:5])
    dif=max_pad_len-mfcc.shape[0]
    #padding sample
    if dif>0:
        #fill holes
        #print("filling holes")
        for i in range(dif):
            mfcc=np.vstack([mfcc,np.zeros(mfcc.shape[1])])
    else:
        #take away excess
        #print("excess")
        mfcc=mfcc[:max_pad_len]
    #print(mfcc.shape)
    #print(mfcc[199,:5])
    return mfcc

# load wav files with this format speaker_word_number.wav
class EasyASR(object):
    '''
    classdocs
    '''
    def __init__(self, filePath="/home/francisco/voz"):
        '''
        Constructor
        '''
        basepath=filePath
        self.mfccs=[]
        self.speakers=[]
        self.words=[]
        fNames=os.listdir(basepath)
        print("fNames=",len(fNames))
        for i,fn in enumerate(fNames):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            mfccs=wav2mfcc(basepath+'/{}.wav'.format(n))
            self.mfccs.append(mfccs)
            #decode file name format speaker_word_number
            speaker,word,number=n.split("_")
            self.speakers.append(speaker)
            self.words.append(word)
        self.leSpeakers=prep.LabelEncoder()
        self.leSpeakers.fit(self.speakers)
        self.iSpeakerLabels=self.leSpeakers.transform(self.speakers)
        self.leWords=prep.LabelEncoder()
        self.leWords.fit(self.words)
        self.iWordLabels=self.leWords.transform(self.words)
        print(len(self.words))
        print(self.words)
        print(self.speakers)

ea=EasyASR()
print(ea.leWords)

Imports doned


FileNotFoundError: [Errno 2] No such file or directory: '/home/francisco/voz'

In [3]:
trainX, trainYwords, trainYspeakers = np.array(ea.mfccs),np.array(ea.iWordLabels),np.array(ea.iSpeakerLabels)
#testX, testY = test

maxlen=trainX.shape[1]
lstm_hidden_units=128
input_dim=trainX.shape[2]
outputWords_dim=max(trainYwords)+1
outputSpeakers_dim=max(trainYspeakers)+1

print(outputWords_dim,outputSpeakers_dim)

# Converting labels to binary vectors
trainYwords_cat    = utils.to_categorical(trainYwords   ,outputWords_dim)
trainYspeakers_cat = utils.to_categorical(trainYspeakers,outputSpeakers_dim)
print (trainX.shape,trainYwords_cat.shape,trainYspeakers_cat.shape)

17 4
(108, 30, 20) (108, 17) (108, 4)


In [7]:
# Network building
def build_model_asr_conv1d(input_dim,outputWords_dim,outputSpeakers_dim,hiden_dim=128):
    input = Input(shape=(30,input_dim))

    # Conv1D + global max pooling
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(input)
    x = Dropout(0.5)(x)
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=2)(x)
    x = Dropout(0.5)(x)
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=2)(x)
    #x = Dropout(0.5)(x)
    #x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    x = GlobalMaxPooling1D()(x)

    # We add a vanilla hidden layer:
    x = Dense(hiden_dim, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(hiden_dim)(x)
    x = Dropout(0.5)(x)
    wd= Dense(hiden_dim)(x)
    sd= Dense(hiden_dim)(x)
    # from: https://www.tensorflow.org/guide/keras/train_and_evaluate
    #       Passing data to multi-input, multi-output models
    prediction_words    = Dense(outputWords_dim   , activation="softmax", name="word")(wd)
    prediction_speakers = Dense(outputSpeakers_dim, activation="softmax", name="speaker")(sd)

    model = Model(input, [prediction_words,prediction_speakers])
    return model

In [5]:
model=build_model_asr_conv1d(input_dim,outputWords_dim,outputSpeakers_dim)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30, 20)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 28, 128)      7808        input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 28, 128)      0           conv1d[0][0]                     
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 13, 128)      49280       dropout[0][0]                    
______________________________________________________________________________________________

In [19]:
# Training
history=model.fit(trainX, [trainYwords_cat,trainYspeakers_cat], batch_size=108,epochs=500,validation_split=0.2)

Train on 86 samples, validate on 22 samples
Epoch 1/500
86/86 [==============================] - 0s 212us/sample - loss: 0.1264 - word_loss: 0.0744 - speaker_loss: 0.0520 - word_accuracy: 0.9767 - speaker_accuracy: 0.9767 - val_loss: 0.6839 - val_word_loss: 0.1680 - val_speaker_loss: 0.5159 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.8636
Epoch 2/500
86/86 [==============================] - 0s 152us/sample - loss: 0.2233 - word_loss: 0.1254 - speaker_loss: 0.0979 - word_accuracy: 0.9651 - speaker_accuracy: 0.9884 - val_loss: 0.6638 - val_word_loss: 0.1589 - val_speaker_loss: 0.5049 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.8636
Epoch 3/500
86/86 [==============================] - 0s 171us/sample - loss: 0.1547 - word_loss: 0.0953 - speaker_loss: 0.0595 - word_accuracy: 0.9767 - speaker_accuracy: 0.9767 - val_loss: 0.6412 - val_word_loss: 0.1535 - val_speaker_loss: 0.4877 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.8636
Epoch 4/500
86/86 [================

Epoch 28/500
86/86 [==============================] - 0s 197us/sample - loss: 0.2600 - word_loss: 0.2529 - speaker_loss: 0.0071 - word_accuracy: 0.9302 - speaker_accuracy: 1.0000 - val_loss: 1.2860 - val_word_loss: 0.2697 - val_speaker_loss: 1.0163 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.8636
Epoch 29/500
86/86 [==============================] - 0s 176us/sample - loss: 0.4186 - word_loss: 0.3565 - speaker_loss: 0.0620 - word_accuracy: 0.9070 - speaker_accuracy: 0.9767 - val_loss: 1.2800 - val_word_loss: 0.2718 - val_speaker_loss: 1.0082 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.8636
Epoch 30/500
86/86 [==============================] - 0s 212us/sample - loss: 0.0977 - word_loss: 0.0609 - speaker_loss: 0.0368 - word_accuracy: 0.9767 - speaker_accuracy: 0.9884 - val_loss: 1.2702 - val_word_loss: 0.2752 - val_speaker_loss: 0.9950 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.8636
Epoch 31/500
86/86 [==============================] - 0s 191us/sample - loss

Epoch 55/500
86/86 [==============================] - 0s 230us/sample - loss: 0.2892 - word_loss: 0.2784 - speaker_loss: 0.0108 - word_accuracy: 0.9186 - speaker_accuracy: 1.0000 - val_loss: 0.8301 - val_word_loss: 0.2454 - val_speaker_loss: 0.5847 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.8636
Epoch 56/500
86/86 [==============================] - 0s 223us/sample - loss: 0.0235 - word_loss: 0.0205 - speaker_loss: 0.0030 - word_accuracy: 1.0000 - speaker_accuracy: 1.0000 - val_loss: 0.8149 - val_word_loss: 0.2465 - val_speaker_loss: 0.5683 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.8636
Epoch 57/500
86/86 [==============================] - 0s 207us/sample - loss: 0.1664 - word_loss: 0.1587 - speaker_loss: 0.0077 - word_accuracy: 0.9535 - speaker_accuracy: 1.0000 - val_loss: 0.8061 - val_word_loss: 0.2477 - val_speaker_loss: 0.5584 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.8636
Epoch 58/500
86/86 [==============================] - 0s 165us/sample - loss

Epoch 82/500
86/86 [==============================] - 0s 184us/sample - loss: 0.1217 - word_loss: 0.0945 - speaker_loss: 0.0271 - word_accuracy: 0.9767 - speaker_accuracy: 0.9884 - val_loss: 1.0270 - val_word_loss: 0.1498 - val_speaker_loss: 0.8772 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.8636
Epoch 83/500
86/86 [==============================] - 0s 175us/sample - loss: 0.1829 - word_loss: 0.1570 - speaker_loss: 0.0259 - word_accuracy: 0.9535 - speaker_accuracy: 1.0000 - val_loss: 0.9534 - val_word_loss: 0.1211 - val_speaker_loss: 0.8323 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.8636
Epoch 84/500
86/86 [==============================] - 0s 214us/sample - loss: 0.1688 - word_loss: 0.1640 - speaker_loss: 0.0049 - word_accuracy: 0.9419 - speaker_accuracy: 1.0000 - val_loss: 0.8986 - val_word_loss: 0.1004 - val_speaker_loss: 0.7982 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.8636
Epoch 85/500
86/86 [==============================] - 0s 167us/sample - loss

Epoch 109/500
86/86 [==============================] - 0s 177us/sample - loss: 0.2443 - word_loss: 0.2084 - speaker_loss: 0.0359 - word_accuracy: 0.9535 - speaker_accuracy: 0.9767 - val_loss: 1.0325 - val_word_loss: 0.1278 - val_speaker_loss: 0.9046 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.8182
Epoch 110/500
86/86 [==============================] - 0s 170us/sample - loss: 0.1758 - word_loss: 0.1717 - speaker_loss: 0.0041 - word_accuracy: 0.9535 - speaker_accuracy: 1.0000 - val_loss: 1.0652 - val_word_loss: 0.1256 - val_speaker_loss: 0.9396 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.8182
Epoch 111/500
86/86 [==============================] - 0s 198us/sample - loss: 0.1430 - word_loss: 0.1255 - speaker_loss: 0.0175 - word_accuracy: 0.9535 - speaker_accuracy: 0.9884 - val_loss: 1.0891 - val_word_loss: 0.1269 - val_speaker_loss: 0.9622 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.8182
Epoch 112/500
86/86 [==============================] - 0s 185us/sample - 

...

Epoch 379/500
86/86 [==============================] - 0s 264us/sample - loss: 0.0618 - word_loss: 0.0540 - speaker_loss: 0.0078 - word_accuracy: 0.9767 - speaker_accuracy: 1.0000 - val_loss: 0.4333 - val_word_loss: 0.1604 - val_speaker_loss: 0.2729 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.9091
Epoch 380/500
86/86 [==============================] - 0s 197us/sample - loss: 0.1407 - word_loss: 0.1332 - speaker_loss: 0.0075 - word_accuracy: 0.9535 - speaker_accuracy: 1.0000 - val_loss: 0.4210 - val_word_loss: 0.1521 - val_speaker_loss: 0.2689 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.9091
Epoch 381/500
86/86 [==============================] - 0s 209us/sample - loss: 0.1594 - word_loss: 0.1400 - speaker_loss: 0.0194 - word_accuracy: 0.9419 - speaker_accuracy: 0.9884 - val_loss: 0.4171 - val_word_loss: 0.1480 - val_speaker_loss: 0.2691 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.9091
Epoch 382/500
86/86 [==============================] - 0s 257us/sample - 

Epoch 406/500
86/86 [==============================] - 0s 201us/sample - loss: 0.1561 - word_loss: 0.0957 - speaker_loss: 0.0604 - word_accuracy: 0.9651 - speaker_accuracy: 0.9884 - val_loss: 0.5783 - val_word_loss: 0.1649 - val_speaker_loss: 0.4134 - val_word_accuracy: 0.9091 - val_speaker_accuracy: 0.8636
Epoch 407/500
86/86 [==============================] - 0s 200us/sample - loss: 0.1277 - word_loss: 0.0891 - speaker_loss: 0.0386 - word_accuracy: 0.9767 - speaker_accuracy: 0.9884 - val_loss: 0.5562 - val_word_loss: 0.1595 - val_speaker_loss: 0.3968 - val_word_accuracy: 0.9091 - val_speaker_accuracy: 0.9091
Epoch 408/500
86/86 [==============================] - 0s 165us/sample - loss: 0.1290 - word_loss: 0.1254 - speaker_loss: 0.0036 - word_accuracy: 0.9651 - speaker_accuracy: 1.0000 - val_loss: 0.5364 - val_word_loss: 0.1514 - val_speaker_loss: 0.3851 - val_word_accuracy: 0.9091 - val_speaker_accuracy: 0.9091
Epoch 409/500
86/86 [==============================] - 0s 168us/sample - 

Epoch 433/500
86/86 [==============================] - 0s 183us/sample - loss: 0.2578 - word_loss: 0.1801 - speaker_loss: 0.0776 - word_accuracy: 0.9651 - speaker_accuracy: 0.9767 - val_loss: 0.4495 - val_word_loss: 0.0226 - val_speaker_loss: 0.4270 - val_word_accuracy: 1.0000 - val_speaker_accuracy: 0.8636
Epoch 434/500
86/86 [==============================] - 0s 166us/sample - loss: 0.2833 - word_loss: 0.1454 - speaker_loss: 0.1379 - word_accuracy: 0.9535 - speaker_accuracy: 0.9767 - val_loss: 0.4080 - val_word_loss: 0.0235 - val_speaker_loss: 0.3845 - val_word_accuracy: 1.0000 - val_speaker_accuracy: 0.9091
Epoch 435/500
86/86 [==============================] - 0s 269us/sample - loss: 0.1944 - word_loss: 0.1618 - speaker_loss: 0.0325 - word_accuracy: 0.9419 - speaker_accuracy: 0.9767 - val_loss: 0.3559 - val_word_loss: 0.0237 - val_speaker_loss: 0.3323 - val_word_accuracy: 1.0000 - val_speaker_accuracy: 0.9091
Epoch 436/500
86/86 [==============================] - 0s 216us/sample - 

Epoch 460/500
86/86 [==============================] - 0s 175us/sample - loss: 0.1489 - word_loss: 0.1014 - speaker_loss: 0.0475 - word_accuracy: 0.9651 - speaker_accuracy: 0.9651 - val_loss: 0.5336 - val_word_loss: 0.0736 - val_speaker_loss: 0.4600 - val_word_accuracy: 0.9545 - val_speaker_accuracy: 0.9091
Epoch 461/500
86/86 [==============================] - 0s 184us/sample - loss: 0.1329 - word_loss: 0.1300 - speaker_loss: 0.0028 - word_accuracy: 0.9535 - speaker_accuracy: 1.0000 - val_loss: 0.5490 - val_word_loss: 0.0883 - val_speaker_loss: 0.4607 - val_word_accuracy: 0.9091 - val_speaker_accuracy: 0.9091
Epoch 462/500
86/86 [==============================] - 0s 168us/sample - loss: 0.1550 - word_loss: 0.1526 - speaker_loss: 0.0024 - word_accuracy: 0.9535 - speaker_accuracy: 1.0000 - val_loss: 0.5611 - val_word_loss: 0.1004 - val_speaker_loss: 0.4607 - val_word_accuracy: 0.9091 - val_speaker_accuracy: 0.9091
Epoch 463/500
86/86 [==============================] - 0s 168us/sample - 

Epoch 487/500
86/86 [==============================] - 0s 275us/sample - loss: 0.2700 - word_loss: 0.1138 - speaker_loss: 0.1561 - word_accuracy: 0.9419 - speaker_accuracy: 0.9651 - val_loss: 0.6975 - val_word_loss: 0.4347 - val_speaker_loss: 0.2628 - val_word_accuracy: 0.9091 - val_speaker_accuracy: 0.9091
Epoch 488/500
86/86 [==============================] - 0s 228us/sample - loss: 0.1825 - word_loss: 0.0770 - speaker_loss: 0.1055 - word_accuracy: 0.9767 - speaker_accuracy: 0.9884 - val_loss: 0.6661 - val_word_loss: 0.4044 - val_speaker_loss: 0.2617 - val_word_accuracy: 0.9091 - val_speaker_accuracy: 0.9091
Epoch 489/500
86/86 [==============================] - 0s 183us/sample - loss: 0.2132 - word_loss: 0.0835 - speaker_loss: 0.1297 - word_accuracy: 0.9651 - speaker_accuracy: 0.9884 - val_loss: 0.6480 - val_word_loss: 0.3769 - val_speaker_loss: 0.2712 - val_word_accuracy: 0.9091 - val_speaker_accuracy: 0.8636
Epoch 490/500
86/86 [==============================] - 0s 211us/sample - 

In [20]:
print(trainX[5:7,:,:].shape)
predWord,predSpeaker=model.predict(trainX[5:6,:,:])
print(predWord,predSpeaker)
y_predWord   = np.argmax(predWord)
y_predSpeaker= np.argmax(predSpeaker)
print(y_predWord,"=",np.max(predWord))
print(y_predSpeaker,"=",np.max(predSpeaker))
word=ea.leWords.inverse_transform([y_predWord])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
print(speaker[0]," ha dicho \'",word[0],"\'")
print(ea.leWords.classes_)
print(ea.leSpeakers.classes_)

(2, 30, 20)
[[1.0566245e-17 1.6993784e-29 1.2629809e-15 8.0412259e-26 3.6855405e-29
  5.2542363e-30 1.8826095e-17 2.3430210e-21 4.9290455e-24 1.1396187e-33
  1.0000000e+00 5.5632385e-27 1.0227323e-27 1.2566632e-27 1.6294543e-26
  3.2783657e-27 8.5621296e-25]] [[1.1922505e-14 7.2233626e-17 6.6164611e-15 1.0000000e+00]]
10 = 1.0
3 = 1.0
paco  ha dicho ' para '
['avanza' 'buenos-dias' 'casa' 'cinco' 'cuatro' 'dos' 'hola' 'no' 'nueve'
 'ocho' 'para' 'retrocede' 'seis' 'si' 'siete' 'tres' 'uno']
['carlos' 'eva' 'irene' 'paco']


In [59]:
!sox -r 16000 -t alsa default recording.wav silence 1 0.1 1% 1 1.5 1%
!aplay "recording.wav"


Input File     : 'default' (alsa)
Channels       : 2
Sample Rate    : 16000
Precision      : 16-bit
Sample Encoding: 16-bit Signed Integer PCM

In:0.00% 00:00:03.33 [00:00:00.00] Out:15.4k [      |      ]        Clip:0    
Done.
Playing WAVE 'recording.wav' : Signed 16 bit Little Endian, Rate 16000 Hz, Stereo


In [24]:
!aplay "recording.wav"

Playing WAVE 'recording.wav' : Signed 16 bit Little Endian, Rate 16000 Hz, Stereo


In [25]:
sample=wav2mfcc("recording.wav")
sample=np.expand_dims(np.array(sample),axis=0)
sample.shape

16000 wave= (3550,) mfcc= (7, 20) file= recording.wav


(1, 30, 20)

In [26]:
predWord,predSpeaker=model.predict([sample])
print(predWord,predSpeaker)
y_predWord   = np.argmax(predWord)
y_predSpeaker= np.argmax(predSpeaker)
print(y_predWord,"=",np.max(predWord))
print(y_predSpeaker,"=",np.max(predSpeaker))
word=ea.leWords.inverse_transform([y_predWord])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
print(speaker[0]," ha dicho \'",word[0],"\'")
print(ea.leWords.classes_)
print(ea.leSpeakers.classes_)

[[9.5779401e-01 5.3499174e-05 3.0562773e-02 6.6761061e-04 1.0508731e-03
  4.9302249e-05 3.6217431e-03 2.6359034e-05 7.7112427e-06 2.2256410e-05
  5.8752643e-03 1.0338744e-04 2.8792516e-05 2.8994787e-06 9.3851040e-06
  1.8573144e-06 1.2219230e-04]] [[5.5516221e-06 7.6552706e-06 1.0617006e-05 9.9997616e-01]]
0 = 0.957794
3 = 0.99997616
paco  ha dicho ' avanza '
['avanza' 'buenos-dias' 'casa' 'cinco' 'cuatro' 'dos' 'hola' 'no' 'nueve'
 'ocho' 'para' 'retrocede' 'seis' 'si' 'siete' 'tres' 'uno']
['carlos' 'eva' 'irene' 'paco']
